<a href="https://colab.research.google.com/github/PauloBoaventura/D._assistente/blob/main/D_assistente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gtts pydub speechrecognition numpy opencv-python yfinance


from gtts import gTTS
from IPython.display import Audio, display, Javascript, Image
import speech_recognition as sr
from pydub import AudioSegment
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import cv2
import os
import yfinance as yf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 32.4 MB/s eta 0:00:00


In [ ]:
def text_to_speech(text, lang='pt', slow=False, mp3_filename='command.mp3', wav_filename='command.wav'):
    try:
        tts = gTTS(text=text, lang=lang, slow=slow)
        tts.save(mp3_filename)

        audio = AudioSegment.from_mp3(mp3_filename)
        audio.export(wav_filename, format="wav")
        return wav_filename
    except Exception as e:
        print(f"Erro ao converter texto em áudio: {e}")
        return None

def recognize_speech_from_file(filename):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = recognizer.record(source)
        try:
            command = recognizer.recognize_google(audio, language='pt-BR')
            print(f"Comando reconhecido: {command}")
            return command.lower()
        except sr.UnknownValueError:
            print("Não entendi o comando.")
            return ""
        except sr.RequestError:
            print("Erro ao conectar com o serviço de reconhecimento de voz.")
            return ""

def open_camera():
    js = Javascript('''
        async function takePhoto() {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            document.body.appendChild(video);
            await new Promise(resolve => setTimeout(resolve, 5000));

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);

            stream.getTracks().forEach(track => track.stop());
            video.remove();

            return canvas.toDataURL('image/jpeg', 0.8);
        }
    ''')
    display(js)
    js_reply = eval_js('takePhoto()')
    image_bytes = b64decode(js_reply.split(',')[1])
    img = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(img, flags=1)
    filename = 'captured_image.jpg'
    cv2.imwrite(filename, img)
    display(Image(filename))
    print(f"Imagem capturada e salva como {filename}.")


In [ ]:

def get_stock_price(symbol):
    try:
        stock = yf.Ticker(symbol)
        price = stock.history(period='1d')['Close'].iloc[-1]
        print(f"O preço da ação {symbol} é R$ {price:.2f}")
    except Exception as e:
        print(f"Erro ao buscar o preço da ação {symbol}: {e}")

def execute_script():
    print("Executando análise financeira...")
    # Aqui pode ser integrado um script real de análise
    print("Análise concluída!")


In [ ]:
if __name__ == "__main__":
    command_text = input("Digite o comando: ")
    audio_file = text_to_speech(command_text)
    if audio_file and os.path.exists(audio_file):
        display(Audio(audio_file))
        command = recognize_speech_from_file(audio_file)

        if 'abrir câmera' in command:
            open_camera()
        elif 'preço da ação' in command:
            symbol = input("Digite o código da ação (ex: PETR4.SA): ")
            get_stock_price(symbol)
        elif 'rodar script de análise financeira' in command:
            execute_script()
        else:
            print("Comando não reconhecido.")
    else:
        print("Falha ao gerar o áudio.")

Digite o comando: preço da ação


Comando reconhecido: preço da ação
Digite o código da ação (ex: PETR4.SA): PETR4.SA
O preço da ação PETR4.SA é R$ 36.28
